In [1]:
import os, time, shutil, tarfile
from minio import Minio
from minio.error import ResponseError
from tqdm import tnrange

times = 1
resultSmallFile = []
testList = [1000]

Gen test file

In [ ]:
with open('file.txt', 'w') as f:
    for i in range(64*1000*1000):
        f.write('a')

os.mkdir('minio_test_data')

for size in testList:
    os.mkdir('minio_test_data/'+str(size))
    for i in range(size):
        shutil.copyfile('file.txt', 'minio_test_data/'+str(size)+'/'+str(i))

Upload test

In [3]:
minioClient = Minio('140.124.181.77:31437',
                    access_key='wei840222',
                    secret_key='j04rmp4jo3',
                    secure=False)

for ti in tnrange(times, desc='times'):
    runTimeListSmallFile = []
    
    for size in testList:
        try:
            minioClient.make_bucket('test-1000'+str(size)+'-'+str(ti), location="us-east-1")
        except ResponseError as err:
            print(err)

        try:
            start = time.time()
            for j in tnrange(size, desc='files'):
                minioClient.fput_object('test-1000'+str(size)+'-'+str(ti), str(j), 'minio_test_data/'+str(size)+'/'+str(j))
            end = time.time()
            runTimeListSmallFile.append(end-start)
        except ResponseError as err:
            print(err)
            print('upload minio_test_data/'+str(size)+'/'+str(j)+'fail')
            runTimeListSmallFile.append(-1)
            
    resultSmallFile.append(runTimeListSmallFile)

output result to csv

In [4]:
with open('small_file_longhorn_1000_upload.csv', 'w') as f:
    f.write(str([str(t)+'-file' for t in testList]).replace("'",'')[1:-1]+'\n')
    for r in resultSmallFile:
        f.write(str(r)[1:-1]+'\n')

Download test

In [5]:
resultSmallFile = []
for ti in tnrange(times, desc='times'):
    runTimeListSmallFile = []
    
    for size in testList:
        try:
            start = time.time()
            for j in tnrange(size, desc='files'):
                minioClient.fget_object('test-1000'+str(size)+'-'+str(ti), str(j), 'minio_test_data_read/'+str(size)+'/'+str(j))
            end = time.time()
            runTimeListSmallFile.append(end-start)
        except ResponseError as err:
            print(err)
            print('download minio_test_data/'+str(size)+'/'+str(j)+'fail')
            runTimeListSmallFile.append(-1)
            
    resultSmallFile.append(runTimeListSmallFile)

In [6]:
with open('small_file_longhorn_1000_download.csv', 'w') as f:
    f.write(str([str(t)+'-file' for t in testList]).replace("'",'')[1:-1]+'\n')
    for r in resultSmallFile:
        f.write(str(r)[1:-1]+'\n')

clean file

In [7]:
try:
    os.remove('file.txt')
except OSError as e:  ## if failed, report it back to the user ##
    print ("Error: %s - %s." % (e.filename, e.strerror))
try:
    shutil.rmtree('minio_test_data')
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))
try:
    shutil.rmtree('minio_test_data_read')
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))